In [64]:
import sys
import warnings
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [10]:
warnings.simplefilter("ignore")

# read

In [12]:
boston = load_boston()

X = pd.DataFrame(data=boston['data'], columns=boston["feature_names"])
y = boston['target']

In [14]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [17]:
y[:5]

array([24. , 21.6, 34.7, 33.4, 36.2])

## よく使うのでpickle保存

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
joblib.dump([X_train, X_test, y_train, y_test], filename='./data/boston_husing.pkl')

['./data/boston_husing.pkl']

# 線形回帰モデル

In [24]:
# 学習
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [31]:
def regression_metrics(model, X, y):
    """ 回帰精度のRMSEとR2をDFとして返す"""
    
    # テストデータで予測
    y_pred = model.predict(X)
    
    # 評価指標をDFにまとめる
    df = pd.DataFrame(
        data={"RMSE":[mean_squared_error(y, y_pred,  squared=False)],
              "R2":[r2_score(y, y_pred)]})
    return df

In [34]:
regression_metrics(lm, X_test, y_test)

,RMSE,R2
0,4.928602,0.668759


## 重要度

In [45]:
def get_coef(model, var_names):
    """ 特徴量と回帰係数が対応したDF作成"""
    
    df = pd.DataFrame(
        data={"coef":[model.intercept_] + model.coef_.tolist()},
        index=['intercept'] + var_names
    )
    
    return df.T

In [54]:
print("各係数")
# 絶対値で見ると NOX（酸化窒素濃度）が最も高いが、重要な説明変数ではない。
# 理由は、説明変数ごとのスケールが異なるため、NOXが高く見えているだけ
display(get_coef(lm, X.columns.tolist()))

print("係数のmax-minの差分")
display(pd.DataFrame(data={'range':X_train.max() - X_train.min()}).T)

各係数


,intercept,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
coef,30.246751,-0.113056,0.03011,0.040381,2.784438,-17.202633,4.438835,-0.006296,-1.447865,0.26243,-0.010647,-0.915456,0.012351,-0.508571


係数のmax-minの差分


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
range,88.96714,100.0,27.0,1.0,0.486,4.917,97.1,10.9969,23.0,524.0,9.4,396.58,36.24


In [60]:
# 標準化して変数のスケールを合わせる

# 訓練データから平均と分散を計算
ss = StandardScaler()
ss.fit(X_train)

# 標準化
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

# 学習
lm_ss = LinearRegression()
lm_ss.fit(X_train_ss, y_train)

#　精度評価 ※標準化しただけなので、結果は変わらない
regression_metrics(lm_ss, X_test_ss, y_test)

,RMSE,R2
0,4.928602,0.668759


In [62]:
# 回帰係数の確認
display(get_coef(lm_ss, X_train.columns.tolist()))

,intercept,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
coef,22.796535,-1.002135,0.696269,0.278065,0.718738,-2.022319,3.14524,-0.176048,-3.081908,2.251407,-1.767014,-2.037752,1.129568,-3.611658


## 予測の理由

In [63]:
# 線形回帰では、回帰係数と説明変数を線形結合すると、簡単に出力可能

In [68]:
# ランダムフォレスト

# 学習
rf = RandomForestRegressor(n_jobs=1, random_state=42)
rf.fit(X_train, y_train)

# モデルの書き出し
joblib.dump(rf, filename="./model/boston_housing_rf.pkl")

# テストデータで精度評価 
# 精度は線形回帰よりも高いが、解釈性が低く、業務で説明しずらい
regression_metrics(rf, X_test, y_test)

,RMSE,R2
0,2.810963,0.892253
